In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

class SoftmaxRegression:
    def __init__(self, lr=0.01, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None
    
    def softmax(self, z):
        exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Tránh tràn số
        return exp_z / np.sum(exp_z, axis=1, keepdims=True)
    
    def fit(self, X, y):
        n_samples, n_features = X.shape
        # Encode string labels to integer indices
        self.classes_, y_indices = np.unique(y, return_inverse=True)
        n_classes = len(self.classes_)
        
        # Khởi tạo trọng số và bias
        self.weights = np.zeros((n_features, n_classes))
        self.bias = np.zeros(n_classes)
        
        # Mã hóa one-hot cho nhãn
        y_one_hot = np.zeros((n_samples, n_classes))
        for i in range(n_samples):
            y_one_hot[i, y_indices[i]] = 1
        
        # Gradient Descent
        for _ in range(self.n_iters):
            # Tính xác suất Softmax
            linear_model = np.dot(X, self.weights) + self.bias
            y_pred = self.softmax(linear_model)
            
            # Tính gradient
            grad_w = (1 / n_samples) * np.dot(X.T, (y_pred - y_one_hot))
            grad_b = (1 / n_samples) * np.sum(y_pred - y_one_hot, axis=0)
            
            # Cập nhật tham số
            self.weights -= self.lr * grad_w
            self.bias -= self.lr * grad_b
            
    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_pred = self.softmax(linear_model)
        class_indices = np.argmax(y_pred, axis=1)
        # Convert indices back to original class labels
        return self.classes_[class_indices]

# Tải và tiền xử lý dữ liệu Iris
iris = pd.read_csv("input_2.csv")
X, y = iris.iloc[:, :-1].values, iris.iloc[:, -1].values

# Chuẩn hóa dữ liệu
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Đọc dữ liệu test từ file csv, đảm bảo đọc đúng format
# Read first few lines to check if there's a header
with open("output_2.csv", "r") as f:
    first_line = f.readline().strip()
    # Check if the first line looks like a header or data
    if ',' in first_line and all(c.isdigit() or c in '.,+-' for c in first_line.replace(',', '')):
        # Looks like data, no header
        X_test = pd.read_csv("output_2.csv", header=None)
    else:
        # Has a header
        X_test = pd.read_csv("output_2.csv")

print(f"Number of test samples: {X_test.shape[0]}")
print(f"X_test data shape: {X_test.shape}")

# Huấn luyện mô hình
model = SoftmaxRegression(lr=0.1, n_iters=1000)
model.fit(X, y)

# Dự đoán trên tất cả mẫu test
# Chuẩn hóa dữ liệu test giống như dữ liệu train
X_test_scaled = scaler.transform(X_test.values)

# Dự đoán nhãn cho dữ liệu test
predictions = model.predict(X_test_scaled)
print(f"Number of predictions: {len(predictions)}")

# Ghi kết quả ra file
with open("iris_predictions.csv", "w") as f:
    f.write("Sample,Predicted_Label\n")
    for i, pred in enumerate(predictions):
        f.write(f"{i+1},{pred}\n")

print("Done writing predictions. Last sample number:", len(predictions))

Number of test samples: 30
X_test data shape: (30, 4)
Number of predictions: 30
Done writing predictions. Last sample number: 30


# Kiến trúc mạng Softmax Regression

## 1. Tổng quan kiến trúc:
- **Input Layer**: n_features đặc trưng đầu vào (4 đặc trưng cho dataset Iris)
- **Linear Layer**: Ma trận trọng số W (4 x 3) + bias b (3)
- **Softmax Activation**: Chuyển đổi logits thành xác suất
- **Output Layer**: 3 lớp tương ứng với 3 loài hoa Iris

## 2. Công thức toán học:
```
z = X @ W + b  # Linear transformation
y_pred = softmax(z)  # Activation function
```

## 3. Kiến trúc chi tiết:
```
Input (150 samples x 4 features)
    ↓
Linear Layer: W(4x3) + b(3)
    ↓  
Logits (150 x 3)
    ↓
Softmax Activation
    ↓
Probabilities (150 x 3)
    ↓
Argmax → Predicted Class
```

# CÁC BƯỚC PHÂN LOẠI TRONG SOFTMAX REGRESSION

## Bước 1: Chuẩn bị dữ liệu
```python
# Input: x = [sepal_length, sepal_width, petal_length, petal_width]
# Ví dụ: x = [5.1, 3.5, 1.4, 0.2]
```

## Bước 2: Linear Transformation (Tính toán tuyến tính)
```
z = X @ W + b
```
- **X**: Ma trận input (n_samples × 4 features)
- **W**: Ma trận trọng số (4 × 3 classes) 
- **b**: Vector bias (3,)
- **z**: Vector logits (3,) - điểm số thô cho mỗi class

## Bước 3: Softmax Activation (Chuyển đổi thành xác suất)
```
y_pred = softmax(z) = exp(z_i) / Σ(exp(z_j))
```
- Chuyển đổi logits thành xác suất [0,1]
- Tổng các xác suất = 1
- Class nào có xác suất cao nhất → Dự đoán

## Bước 4: Prediction (Đưa ra dự đoán)
```
predicted_class = argmax(y_pred)
```
- Chọn class có xác suất cao nhất
- Trả về tên class (setosa, versicolor, virginica)

## Ví dụ cụ thể:
```
Input: [5.1, 3.5, 1.4, 0.2]
↓
Logits: [-2.1, 0.8, -1.3]
↓
Softmax: [0.05, 0.89, 0.06]
↓
Prediction: versicolor (index=1, confidence=89%)
```

# Giải thích hàm Softmax

## Công thức Softmax:
```
softmax(z_i) = exp(z_i) / Σ(exp(z_j)) for j=1 to K
```

## Tại sao cần trừ max(z)?
- **Vấn đề**: `exp(z)` có thể gây tràn số với z lớn
- **Giải pháp**: `exp(z - max(z))` giữ nguyên tỷ lệ nhưng tránh tràn số
- **Ví dụ**: 
  - z = [1000, 1001, 1002] → exp(z) = overflow!
  - z - max(z) = [-2, -1, 0] → exp(z-max) = [0.135, 0.368, 1.0] ✓

## Đặc tính của Softmax:
1. **Tổng bằng 1**: Σ softmax(z_i) = 1
2. **Giá trị [0,1]**: Mỗi output là xác suất hợp lệ
3. **Phân biệt tốt**: Làm nổi bật class có logit cao nhất
4. **Differentiable**: Có thể tính gradient cho training

## Code implementation:
```python
def softmax(self, z):
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # Tránh tràn số
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)
```

In [ ]:
# Demo chi tiết hàm Softmax với ví dụ cụ thể
print("=== DEMO HÀM SOFTMAX ===\n")

# Ví dụ với logits từ demo trên
logits = np.array([[8.44541737, 3.78114046, -12.22655782]])
print(f"1. Logits ban đầu: {logits[0]}")

# Bước 1: Trừ max để tránh tràn số
max_logit = np.max(logits, axis=1, keepdims=True)
print(f"2. Max logit: {max_logit[0][0]:.2f}")

stable_logits = logits - max_logit
print(f"3. Logits sau khi trừ max: {stable_logits[0]}")

# Bước 2: Tính exp
exp_logits = np.exp(stable_logits)
print(f"4. exp(logits): {exp_logits[0]}")

# Bước 3: Tính tổng
sum_exp = np.sum(exp_logits, axis=1, keepdims=True)
print(f"5. Tổng exp: {sum_exp[0][0]:.6f}")

# Bước 4: Chia để có xác suất
softmax_probs = exp_logits / sum_exp
print(f"6. Xác suất Softmax: {softmax_probs[0]}")
print(f"7. Tổng xác suất: {np.sum(softmax_probs[0]):.6f}")

# So sánh với hàm softmax của model
model_softmax = model.softmax(logits)
print(f"\n8. Kết quả từ hàm model: {model_softmax[0]}")
print(f"9. Kiểm tra giống nhau: {np.allclose(softmax_probs, model_softmax)}")

print("\n=== GIẢI THÍCH ===\n")
print("• Trừ max(z) giúp tránh tràn số khi exp() với số lớn")
print("• Softmax biến đổi logits thành phân phối xác suất")
print("• Class có logit cao nhất sẽ có xác suất cao nhất")
print("• Tổng tất cả xác suất luôn bằng 1")

In [2]:
# Demo quá trình phân loại với 1 mẫu cụ thể
print("=== DEMO QUÁ TRÌNH PHÂN LOẠI ===")

# Lấy 1 mẫu từ dữ liệu test
sample_idx = 0
x_sample = X_test_scaled[sample_idx:sample_idx+1]  # Shape: (1, 4)
print(f"\n1. Input features: {x_sample[0]}")

# Bước 1: Linear transformation
linear_output = np.dot(x_sample, model.weights) + model.bias
print(f"\n2. Linear output (logits): {linear_output[0]}")
print(f"   Shape: {linear_output.shape}")

# Bước 2: Softmax activation  
softmax_output = model.softmax(linear_output)
print(f"\n3. Softmax probabilities: {softmax_output[0]}")
print(f"   Sum of probabilities: {np.sum(softmax_output[0]):.6f}")

# Bước 3: Prediction (argmax)
predicted_class_idx = np.argmax(softmax_output[0])
predicted_class = model.classes_[predicted_class_idx]
print(f"\n4. Predicted class index: {predicted_class_idx}")
print(f"   Predicted class name: {predicted_class}")
print(f"   Confidence: {softmax_output[0][predicted_class_idx]:.4f}")

# So sánh với các class khác
print("\n=== CONFIDENCE CHO TẤT CẢ CLASSES ===")
for i, class_name in enumerate(model.classes_):
    prob = softmax_output[0][i]
    status = "← PREDICTED" if i == predicted_class_idx else ""
    print(f"{class_name}: {prob:.4f} {status}")

=== DEMO QUÁ TRÌNH PHÂN LOẠI ===

1. Input features: [-0.77168636  2.08046089 -1.80095818 -1.79182202]

2. Linear output (logits): [  8.44541737   3.78114046 -12.22655782]
   Shape: (1, 3)

3. Softmax probabilities: [9.90661958e-01 9.33804069e-03 1.04279930e-09]
   Sum of probabilities: 1.000000

4. Predicted class index: 0
   Predicted class name: Iris-setosa
   Confidence: 0.9907

=== CONFIDENCE CHO TẤT CẢ CLASSES ===
Iris-setosa: 0.9907 ← PREDICTED
Iris-versicolor: 0.0093 
Iris-virginica: 0.0000 


# Quá trình Training (Gradient Descent)

## 1. Loss Function - Cross Entropy:
```
Loss = -Σ y_true * log(y_pred)
```

## 2. Gradient Descent:
```python
# Forward pass
z = X @ W + b
y_pred = softmax(z)

# Compute gradients
grad_W = (1/n) * X.T @ (y_pred - y_true)
grad_b = (1/n) * Σ(y_pred - y_true)

# Update parameters
W = W - lr * grad_W
b = b - lr * grad_b
```

## 3. One-hot Encoding:
- **Input**: ['setosa', 'versicolor', 'virginica']
- **One-hot**: [[1,0,0], [0,1,0], [0,0,1]]
- **Mục đích**: Chuyển nhãn thành vector để tính loss

## 4. Tại sao Softmax phù hợp cho multi-class?
- **Output**: Phân phối xác suất trên tất cả classes
- **Interpretable**: Mỗi giá trị là xác suất thuộc class đó
- **Differentiable**: Có thể tính gradient để training